In [1]:
BASE_DIR = '/content/vae/'
!mkdir -p $BASE_DIR
%cd $BASE_DIR
!wget -c -nc -q http://nlp.stanford.edu/data/glove.6B.zip
!unzip -n glove.6B.zip glove.6B.300d.txt 
!apt-get -yq install -q jq
!wget -c -nc -q http://images.cocodataset.org/annotations/annotations_trainval2014.zip
!unzip -p annotations_trainval2014.zip annotations/captions_train2014.json > captions_train2014.json
!jq -r '.annotations[]|.caption' < captions_train2014.json > sentences.txt
!unzip -p annotations_trainval2014.zip annotations/captions_val2014.json > captions_val2014.json
!jq -r '.annotations[]|.caption' < captions_val2014.json >> sentences.txt


/content/vae
Archive:  glove.6B.zip


In [2]:
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
from keras.layers import Bidirectional, Dense, Embedding, Input, Lambda, LSTM, RepeatVector, TimeDistributed, Layer, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.layers.advanced_activations import ELU
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam, Nadam
from keras import backend as K
import matplotlib.pyplot as plt
from keras.models import Model
from scipy import spatial
import tensorflow as tf
import pandas as pd
import numpy as np
import codecs
import csv
import os

Using TensorFlow backend.


### Directories and text loading
Initially we will set the main directories and some variables regarding the characteristics of our texts.
We set the maximum sequence length to 25, the maximun number of words in our vocabulary to 12000 and we will use 300-dimensional embeddings. Finally we load our texts from a csv. The text file is the train file of the Quora Kaggle challenge containing around 808000 sentences.

In [4]:

EMBEDDING_DIM = 300
TRAIN_DATA_FILE = BASE_DIR + 'sentences.txt'
GLOVE_EMBEDDING = BASE_DIR + 'glove.6B.{0}d.txt'.format(EMBEDDING_DIM)
MODEL_DIR='/content/gdrive/My Drive/Colab Notebooks'
VALIDATION_SPLIT = 0.2
MAX_SEQUENCE_LENGTH = 20
MAX_NB_WORDS = 2000
MAX_TRAIN=70000
MAX_VALID=8000

texts = [] 
with open(TRAIN_DATA_FILE) as f:
    for line in f:
        text = line[0:-1]
        if len(text.split()) <= MAX_SEQUENCE_LENGTH:
                texts.append(text)
print('Found %s texts in train.csv' % len(texts))
n_sents = len(texts)


Found 615599 texts in train.csv


### Text Preprocessing
To preprocess the text we will use the tokenizer and the text_to_sequences function from Keras


In [5]:
tokenizer = Tokenizer(MAX_NB_WORDS+1, oov_token='unk') #+1 for 'unk' token
tokenizer.fit_on_texts(texts)
print('Found %s unique tokens' % len(tokenizer.word_index))
## **Key Step** to make it work correctly otherwise drops OOV tokens anyway!
tokenizer.word_index = {e:i for e,i in tokenizer.word_index.items() if i <= MAX_NB_WORDS} # <= because tokenizer is 1 indexed
tokenizer.word_index[tokenizer.oov_token] = MAX_NB_WORDS + 1
word_index = tokenizer.word_index #the dict values start from 1 so this is fine with zeropadding
index2word = {v: k for k, v in word_index.items()}
sequences = tokenizer.texts_to_sequences(texts)
trimmed_texts = []
trimmed_sequences = []
for i in range(len(sequences)):
    fail = False
    for enc in sequences[i]:
        if enc >= MAX_NB_WORDS:
            fail = True
    if not fail:
        trimmed_texts.append(texts[i])
        trimmed_sequences.append(sequences[i])
texts = trimmed_texts
sequences = trimmed_sequences
print('Text sequences reduced to: {}'.format(len(sequences)))
data_1 = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', data_1.shape)
NB_WORDS = (min(tokenizer.num_words, len(word_index))+1) #+1 for zero padding 

data_val = data_1[MAX_TRAIN:MAX_TRAIN+MAX_VALID]
data_train = data_1[:MAX_TRAIN]

Found 27756 unique tokens
Text sequences reduced to: 424750
Shape of data tensor: (424750, 20)


### Word embeddings
We will use pretrained Glove word embeddings as embeddings for our network. We create a matrix with one embedding for every word in our vocabulary and then we will pass this matrix as weights to the keras embedding layer of our model

In [6]:
embeddings_index = {}
f = open(GLOVE_EMBEDDING, encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

glove_embedding_matrix = np.zeros((NB_WORDS, EMBEDDING_DIM))
for word, i in word_index.items():
    if i < NB_WORDS: #+1 for 'unk' oov token
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            glove_embedding_matrix[i] = embedding_vector
        else:
            # words not found in embedding index will the word embedding of unk
            glove_embedding_matrix[i] = embeddings_index.get('unk')
print('Null word embeddings: %d' % np.sum(np.sum(glove_embedding_matrix, axis=1) == 0))

Found 400000 word vectors.
Null word embeddings: 2


### VAE model
Our model is based on a seq2seq architecture with a bidirectional LSTM encoder and an LSTM decoder and ELU activations.
We feed the latent representation at every timestep as input to the decoder through "RepeatVector(max_len)".
To avoid the one-hot representation of labels we use the "tf.contrib.seq2seq.sequence_loss" that requires as labels only the word indexes (the same that go in input to the embedding matrix) and calculates internally the final softmax (so the model ends with a dense layer with linear activation). Optionally the "sequence_loss" allows to use the sampled softmax which helps when dealing with large vocabularies (for example with a 50k words vocabulary) but in this I didn't use it. The decoder that we are using here is different from the one implemented in the paper; instead of feeding the context vector as initial state of the decoder and the predicted words as inputs, we are feeding the latent representation z as input at every timestep.

In [11]:
batch_size = 100
max_epochs = 100
max_len = MAX_SEQUENCE_LENGTH
emb_dim = EMBEDDING_DIM
latent_dim = 128
intermediate_dim = 512
epsilon_std = 1.0
kl_weight = 0.01
num_sampled=24
act = ELU()


x = Input(shape=(max_len,))
x_embed = Embedding(NB_WORDS, emb_dim, weights=[glove_embedding_matrix],
                            input_length=max_len, trainable=False)(x)
h = Bidirectional(LSTM(intermediate_dim, return_sequences=False, recurrent_dropout=0.2), merge_mode='concat')(x_embed)
#h = Bidirectional(LSTM(intermediate_dim, return_sequences=False), merge_mode='concat')(h)
#h = Dropout(0.2)(h)
#h = Dense(intermediate_dim, activation='linear')(h)
#h = act(h)
#h = Dropout(0.2)(h)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
# we instantiate these layers separately so as to reuse them later
repeated_context = RepeatVector(max_len)
decoder_h = LSTM(intermediate_dim, return_sequences=True, recurrent_dropout=0.2)
decoder_mean = Dense(NB_WORDS, activation='linear')#softmax is applied in the seq2seqloss by tf #TimeDistributed()
h_decoded = decoder_h(repeated_context(z))
x_decoded_mean = decoder_mean(h_decoded)


# placeholder loss
def zero_loss(y_true, y_pred):
    return K.zeros_like(y_pred)

print('y')
#Sampled softmax
logits = tf.constant(np.random.randn(batch_size, max_len, NB_WORDS), tf.float32)
targets = tf.constant(np.random.randint(NB_WORDS, size=(batch_size, max_len)), tf.int32)
proj_w = tf.constant(np.random.randn(EMBEDDING_DIM, NB_WORDS), tf.float32)
proj_b = tf.constant(np.zeros(NB_WORDS), tf.float32)
print('x')

def _sampled_loss(labels, logits):
    labels = tf.cast(labels, tf.int64)
    labels = tf.reshape(labels, [-1, 1])
    logits = tf.cast(logits, tf.float32)
    return tf.cast(
                    tf.nn.sampled_softmax_loss(
                        proj_w,
                        proj_b,
                        labels,
                        logits,
                        num_sampled=num_sampled,
                        num_classes=NB_WORDS),
                    tf.float32)
softmax_loss_f = _sampled_loss


# Custom loss layer
class CustomVariationalLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)
        self.target_weights = tf.constant(np.ones((batch_size, max_len)), tf.float32)

    def vae_loss(self, x, x_decoded_mean):
        #xent_loss = K.sum(metrics.categorical_crossentropy(x, x_decoded_mean), axis=-1)
        labels = tf.cast(x, tf.int32)
        xent_loss = K.sum(tf.contrib.seq2seq.sequence_loss(x_decoded_mean, labels, 
                                                     weights=self.target_weights,
                                                     average_across_timesteps=False,
                                                     average_across_batch=False,#), axis=-1)#,
                                                     softmax_loss_function=softmax_loss_f), axis=-1)#,
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        xent_loss = K.mean(xent_loss)
        kl_loss = K.mean(kl_loss)
        return K.mean(xent_loss + kl_weight * kl_loss)

    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        print(x.shape, x_decoded_mean.shape)
        loss = self.vae_loss(x, x_decoded_mean)
        self.add_loss(loss, inputs=inputs)
        # we don't use this output, but it has to have the correct shape:
        return K.ones_like(x)
    
def kl_loss(x, x_decoded_mean):
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    kl_loss = kl_weight * kl_loss
    return kl_loss

loss_layer = CustomVariationalLayer()([x, x_decoded_mean])
vae = Model(x, [loss_layer])
opt = Adam(lr=0.005) 
opt = Nadam(lr=0.001) 
vae.compile(optimizer=opt, loss=[zero_loss], metrics=[kl_loss])
vae.summary()

y
x
(?, 20) (?, 20, 2001)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 20, 300)      600300      input_3[0][0]                    
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 1024)         3330048     embedding_3[0][0]                
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 128)          131200      bidirectional_3[0][0]            
___________________________________________________________________________________

### Model training
We train our model for 100 epochs through keras ".fit()". For validation data we pass the same array twice since input and labels of this model are the same. If we didn't use the "tf.contrib.seq2seq.sequence_loss" (or another similar function) we would have had to pass as labels the sequence of word one-hot encodings with dimension (batch_size, seq_len, vocab_size) consuming a lot of memory.

In [0]:
def create_model_checkpoint(dir, model_name):
    filepath = dir + '/' + model_name + ".h5" 
    directory = os.path.dirname(filepath)
    try:
        os.stat(directory)
    except:
        os.mkdir(directory)
    checkpointer = ModelCheckpoint(filepath=filepath, verbose=1, save_best_only=True)
    return checkpointer

checkpointer = create_model_checkpoint(MODEL_DIR, 'vae_seq2seq_test_very_high_std')



history = vae.fit(data_train, data_train,
     shuffle=True,
     epochs=max_epochs,
     batch_size=batch_size,
     verbose=2,
     validation_data=(data_val, data_val), callbacks=[checkpointer])

#print(K.eval(vae.optimizer.lr))
#K.set_value(vae.optimizer.lr, 0.01)

vae.save(MODEL_DIR + '/vae_lstm.h5')
#vae.load_weights('models/vae_seq2seq_test.h5')

Train on 70000 samples, validate on 8000 samples
Epoch 1/100
 - 142s - loss: 51.3716 - kl_loss: 1.0872 - val_loss: 38.2140 - val_kl_loss: 1.1813

Epoch 00001: val_loss improved from inf to 38.21398, saving model to /content/gdrive/My Drive/Colab Notebooks/vae_seq2seq_test_very_high_std.h5
Epoch 2/100
 - 137s - loss: 26.7506 - kl_loss: 1.2438 - val_loss: 24.7833 - val_kl_loss: 1.3165

Epoch 00002: val_loss improved from 38.21398 to 24.78333, saving model to /content/gdrive/My Drive/Colab Notebooks/vae_seq2seq_test_very_high_std.h5
Epoch 3/100
 - 137s - loss: 23.6589 - kl_loss: 1.3237 - val_loss: 23.1875 - val_kl_loss: 1.4181

Epoch 00003: val_loss improved from 24.78333 to 23.18754, saving model to /content/gdrive/My Drive/Colab Notebooks/vae_seq2seq_test_very_high_std.h5
Epoch 4/100
 - 137s - loss: 38.3874 - kl_loss: 1.2906 - val_loss: 47.2712 - val_kl_loss: 0.7300

Epoch 00004: val_loss did not improve from 23.18754
Epoch 5/100
 - 138s - loss: 38.1599 - kl_loss: 0.9397 - val_loss: 28.

In [0]:
!ls -sh models/vae_seq2seq_test_very_high_std.h5
!ls -sh models/vae_lstm.h5

In [0]:
if history != None:
  # summarize history for accuracy
  plt.plot(history.history['kl_loss'])
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_kl_loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model val loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['kl_loss', 'loss'], loc='upper left')
  plt.show()

### Project and sample sentences from the latent space
Now we build an encoder model model that takes a sentence and projects it on the latent space and a decoder model that goes from the latent space back to the text representation

In [0]:
# build a model to project inputs on the latent space
encoder = Model(x, z_mean)
#encoder.save('models/encoder32dim512hid30kvocab_loss29_val34.h5')

# build a generator that can sample from the learned distribution
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(repeated_context(decoder_input))
_x_decoded_mean = decoder_mean(_h_decoded)
_x_decoded_mean = Activation('softmax')(_x_decoded_mean)
generator = Model(decoder_input, _x_decoded_mean)

### Test on validation sentences

In [0]:
index2word = {v: k for k, v in word_index.items()}
index2word[0] = 'pad'

#test on a validation sentence
sent_idx = 100
sent_encoded = encoder.predict(data_val[sent_idx:sent_idx+2,:])
x_test_reconstructed = generator.predict(sent_encoded, batch_size = 1)
reconstructed_indexes = np.apply_along_axis(np.argmax, 1, x_test_reconstructed[0])
#np.apply_along_axis(np.max, 1, x_test_reconstructed[0])
#np.max(np.apply_along_axis(np.max, 1, x_test_reconstructed[0]))
word_list = list(np.vectorize(index2word.get)(reconstructed_indexes))
print(' '.join(word_list))
original_sent = list(np.vectorize(index2word.get)(data_val[sent_idx]))
print(' '.join(original_sent))

### Sentence processing and interpolation

In [0]:
# function to parse a sentence
def sent_parse(sentence, mat_shape):
    sequence = tokenizer.texts_to_sequences(sentence)
    padded_sent = pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH)
    return padded_sent#[padded_sent, sent_one_hot]

# input: encoded sentence vector
# output: encoded sentence vector in dataset with highest cosine similarity
def find_similar_encoding(sent_vect):
    all_cosine = []
    for sent in sent_encoded:
        result = 1 - spatial.distance.cosine(sent_vect, sent)
        all_cosine.append(result)
    data_array = np.array(all_cosine)
    maximum = data_array.argsort()[-3:][::-1][1]
    new_vec = sent_encoded[maximum]
    return new_vec

# input: two points, integer n
# output: n equidistant points on the line between the input points (inclusive)
def shortest_homology(point_one, point_two, num):
    dist_vec = point_two - point_one
    sample = np.linspace(0, 1, num, endpoint = True)
    hom_sample = []
    for s in sample:
        hom_sample.append(point_one + s * dist_vec)
    return hom_sample

# input: original dimension sentence vector
# output: sentence text
def print_latent_sentence(sent_vect):
    sent_vect = np.reshape(sent_vect,[1,latent_dim])
    sent_reconstructed = generator.predict(sent_vect)
    sent_reconstructed = np.reshape(sent_reconstructed,[max_len,NB_WORDS])
    reconstructed_indexes = np.apply_along_axis(np.argmax, 1, sent_reconstructed)
    word_list = list(np.vectorize(index2word.get)(reconstructed_indexes))
    w_list = [w for w in word_list if w not in ['pad']]
    print(' '.join(w_list))
    #print(word_list)
     
def new_sents_interp(sent1, sent2, n):
    tok_sent1 = sent_parse(sent1, [MAX_SEQUENCE_LENGTH + 2])
    tok_sent2 = sent_parse(sent2, [MAX_SEQUENCE_LENGTH + 2])
    enc_sent1 = encoder.predict(tok_sent1, batch_size = 16)
    enc_sent2 = encoder.predict(tok_sent2, batch_size = 16)
    test_hom = shortest_homology(enc_sent1, enc_sent2, n)
    for point in test_hom:
        print_latent_sentence(point)

### Example
Now we can try to parse two sentences and interpolate between them generating new sentences

In [0]:
sentence1=['gogogo a woman on a bicyle endend']
mysent = sent_parse(sentence1, [MAX_SEQUENCE_LENGTH + 2])
mysent_encoded = encoder.predict(mysent, batch_size = 16)
print_latent_sentence(mysent_encoded)
print_latent_sentence(find_similar_encoding(mysent_encoded))

sentence2=['gogogo many people running towards a stop sign endend']
mysent2 = sent_parse(sentence2, [MAX_SEQUENCE_LENGTH + 2])
mysent_encoded2 = encoder.predict(mysent2, batch_size = 16)
print_latent_sentence(mysent_encoded2)
print_latent_sentence(find_similar_encoding(mysent_encoded2))
print('-----------------')

new_sents_interp(sentence1, sentence2, 5)

### Results
TO UPDATE

Results are not yet completely satisfying because not all the sentences are grammatically correct and in the interpolation the same sentence has been generated multiple times but anyway the model, even in this preliminary version seems to start working.
There are certainly many improvements that could be done like:
-  parameter tuning (this model trains in few hours on a GTX950M with 2GB memory so it's definitely possible to try larger nets)
-  train on a more general dataset (Quora sentences are all questions)